In [16]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

DATAPATH = "../data"

In [17]:
df1 = pd.read_csv(os.path.join(DATAPATH, "osm_competition.csv")) # OSM data from the series4 competition (including competition results)
df2 = pd.read_csv(os.path.join(DATAPATH, "evariste_nov21.csv")) # New experimental data Evariste from November 2021
df3 = pd.read_csv(os.path.join(DATAPATH, "evariste_mar22.csv")) # New experimental data Evariste from March 2022
df4 = pd.read_csv(os.path.join(DATAPATH, "ersilia_nov22.csv")) # New experimental data Ersilia from November 2022
df5 = pd.read_csv(os.path.join(DATAPATH, "evariste_feb23.csv")) # New experimental data Evariste from February 2023
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
df.to_csv(os.path.join(DATAPATH, "osm_all.csv"), index=False) # All experimental data available for OSM

In [18]:
# create train and test files for ZairaChem
def binarize(df, cutoff):
    bin = []
    for v in df["activity"]:
        if v <= cutoff:
            b = 1
        else:
            b = 0
        bin += [b]
    df["bin"] = bin
    return df

def random_split(df, size):
    indices = np.arange(len(df))
    X_train, X_test, y_train, y_test, i_train, i_test = train_test_split(df["smiles"], df["activity"], indices, test_size=size, stratify=df["bin"])
    train = df.iloc[i_train]
    test = df.iloc[i_test]
    return train, test


In [20]:
CUTOFF = 2.5
#create 5 splits for fold validation of the zairachem models:

for i in np.arange(1,6):
    df_bin = binarize(df, CUTOFF)
    train, test = random_split(df_bin, 0.2)
    train.to_csv(os.path.join(DATAPATH, "train_test_splits", "osm_train{}_{}.csv".format(i, CUTOFF)), index=False)
    test.to_csv(os.path.join(DATAPATH, "train_test_splits", "osm_test{}_{}.csv".format(i, CUTOFF)), index=False)


# Model Performance
Model performance using ZairaChem 